In [1]:
# Simplified experimental setup, we will try to experiment with one/more additional crossover operator. We insert our own tau function as well

In [2]:
# Imports
import numpy as np
import inspect
from queue import PriorityQueue

In [9]:
# Vertex data object: identified by name
class Vertex:
    def __init__(self, name):
        self.name = name
        
# Edge data object: edge (frm -> to) with weight function 'tau'
class Edge:
    def __init__(self, frm, to, tau, dijkstra_weight=1):
        self.frm = frm
        self.to = to
        self.tau = tau
        self.dijkstra_weight = dijkstra_weight
        
    # made dnorm absolute value to prevent negative weights. Is this ok?    
    def generate_dijkstra_weight(self, k):
        self.dijkstra_weight = abs(np.random.normal(self.tau(k), 0.8*self.tau(k)))
        return self.dijkstra_weight
        
    def __repr__(self):
        tau_str = str(inspect.getsourcelines(self.tau)[0])
        tau_str = tau_str.strip("['\\n']").split(" = ")[1]
        return f'\n({self.frm.name} -> {self.to.name}, {tau_str})'

# Graph data object
class Graph:
    def __init__(self):
        self.vertices = []
        self.edges = []
        
    def add_vertex(self, name):
        self.vertices.append(Vertex(name))
        
    def add_vertices(self, num_vertices):
        for i in range(num_vertices):
            self.add_vertex(i+1)
            
    def get_vertex(self, name):
        return next(v for v in self.vertices if v.name == name)
    
    def get_edge(self, v1_name, v2_name):
        for e in self.edges:
            if e.frm.name == v1_name and e.to.name == v2_name:
                return e
            elif e.to.name == v1_name and e.frm.name == v2_name:
                return e
#             else:
#                 print('no edge found ', v1_name, ' ', v2_name)
        
        
    def add_edge(self, frm_name, to_name, tau):
        assert frm_name in [v.name for v in self.vertices], f'Vertex {frm} not in Graph'
        assert to_name in [v.name for v in self.vertices], f'Vertex {to} not in Graph'
        frm = self.get_vertex(frm_name)
        to = self.get_vertex(to_name)
        self.edges.append(Edge(frm, to, tau))
        
    def get_neighbors(self, vertex_name):
        assert vertex_name in [v.name for v in self.vertices], f'Vertex {vertex_name} not in Graph'
        neighbors = []
        for edge in self.edges:
            if edge.frm.name == vertex_name:
                neighbors.append(edge.to)
            if edge.to.name == vertex_name:
                neighbors.append(edge.frm)
        return neighbors
    
    def generate_dijkstra_weights(self, k):
        for edge in self.edges:
            edge.generate_dijkstra_weight(k)
    
    def __str__(self):
        return str(self.edges)

In [4]:
# Test graph parameters
tau_linear = lambda x: x
tau_constant = lambda x: 2
num_vertices = 7
graph_test = Graph()

# Add vertices
graph_test.add_vertices(8)

# Add edges
graph_test.add_edge(1, 2, tau_linear)
graph_test.add_edge(1, 3, tau_linear)
graph_test.add_edge(1, 4, tau_linear)
graph_test.add_edge(2, 6, tau_linear)
graph_test.add_edge(3, 4, tau_linear)
graph_test.add_edge(3, 8, tau_linear)
graph_test.add_edge(4, 5, tau_linear)
graph_test.add_edge(5, 8, tau_linear)
graph_test.add_edge(6, 7, tau_linear)
graph_test.add_edge(7, 8, tau_linear)



In [7]:
# RandDijkstra
# k is continuous flow of drivers (real number > 0)
def rand_dijkstra(graph, source, target, k):
#     graph.generate_dijkstra_weights(k)
    
    route_to_target = []
    D = {v.name:float('inf') for v in (graph.vertices)}
    D[source.name] = 0
    visited = []

    pq = PriorityQueue()
    pq.put((0, source.name))

    while not pq.empty():
        (dist, current_vertex_name) = pq.get()
        visited.append(current_vertex_name)

        for neighbor in graph.get_neighbors(current_vertex_name):
            if neighbor not in visited:
                edge = graph.get_edge(current_vertex_name, neighbor.name)
                weight = edge.generate_dijkstra_weight(k)
                old_cost = D[neighbor.name]
                new_cost = D[current_vertex_name] + weight
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor.name))
                    D[neighbor.name] = new_cost
                    
    current_node_name = target.name
    route_to_target.append(target.name)
    while source.name not in route_to_target:   
        shortest = (float('inf'), 0)
        for neighbor in graph.get_neighbors(current_node_name):
            if D[neighbor.name] < shortest[0]:
                shortest = (D[neighbor.name], neighbor.name)
        route_to_target.append(shortest[1])
        current_node_name = shortest[1]
        
    route_to_target.reverse()
    return D[target.name], route_to_target

print(rand_dijkstra(graph_test, graph_test.get_vertex(1), graph_test.get_vertex(8), 100))

(276.7619164937433, [1, 3, 8])


In [10]:
#mutation operators: tuple of 4 operator functions
def multiple_router_EA(G, s, t, k, n, mu, cStra=lambda x, y: x, criterion=False):
    P = []
    ex_segment_index = 3 # index of exSegment operator
    
    for i in range(mu):
        ind = [rand_dijkstra(G, s, t, k)[1] for _ in range(n)]
        P.append(ind)
    while not criterion:
        C = []
        for j in range(int(sqrt(mu**2-(mu/2)))):
            # choose uniformly at random chosen
            inds = np.random.choice(P, 2, replace=False)
            ind1 = inds[0]
            ind2 = inds[1]
            
            C.append(cStra(ind1, ind2))
        P_ = P.copy()
        for ind in P_:
            mutations = max(1, np.random.poisson(1.5))
            ops_indices = []
            for j in range(mutations):
                operator_index = np.random(randint(len(mutation_operators)))
                ops_indices.append(operator_index)
                
            # Because exSegment is too expense: if it is in the list
            # we will discard all other operators
            if ex_segment_index is in ops_indices:
                ops_indices = [ex_segment_index]
            
            # returns a list of mutation operators when given a list of
            # integers
            ops = get_operators(ops_indices)
            
            for operator in ops:
                #TBI
            
        # if no individual in 𝐶 is better than the best in 𝑃 then:
            #C = []
        
        #P = best mu individuals in C U P_ U P 
    
    return best_P
        
        

                
def get_operators(indices):
    operators = []
    for index in indices:        
        if index == 0:
            operators.append(newRoute())
            
        elif index == 1:
            operators.append(randomP())
            
        elif index == 2:
            operators.append(linkWP())
            
        elif index == 3:
            operators.append(exSegment())
            
        elif index == 4:
            operators.append(OurCustomOperator())

SyntaxError: invalid syntax (Temp/ipykernel_19076/2596390466.py, line 28)

In [15]:
# Traffic Flow (f) = list of integers mapping index of route to drivers per unit

# Helper function that retrieves all edges in a route
def get_edges(graph, route):
    edges = []
    for i in range(len(route)-1):
        edges.append(graph.get_edge(route[i], route[i+1]))
    return edges

# Helper function to get traffic flow of edge given routes (P_st), traffic flow (f) and edge
def get_edge_traffic_flow(P_st, f, edge):
    value = 0
    for i, p in enumerate(P_st):
        for i in range(len(p)-1):
            if (p[i] == edge.frm.name and p[i+1] == edge.to.name) or (p[i+1] == edge.frm.name and p[i] == edge.to.name):
                value += f[i]
                break
    return value

# Function to calculate travel time
def calculate_travel_time(P_st, f, p, i, graph):
    f_p = f[i]
    tau_p = [edge.tau(get_edge_traffic_flow(P_st, f, edge)) for edge in get_edges(graph, p)]
    return f_p * tau_p

# Function to calculate overall travel time (C)
def calculate_overall_travel_time(P_st, f, graph):
    overall_travel_time = 0
    for i, p in enumerate(P_st):
        overall_travel_time += calculate_travel_time(P_st, f, p, i, graph)
    return overall_travel_time

In [ ]:
# Mutation operator New Route
def new_route(P_st):
    pass